Who is a similar author?
- An author who writes articles from similar categories
- An author who writes from same languages
    
Filters to keep:
- recent article
- more liked/viewed/commented article
- irrespective of an article being a challenge or not we can take a user's categories as those categories that have acquired more impressions and based on the categories of user we can recomend articles


In [6]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re


from elasticsearch import Elasticsearch
import elasticsearch.helpers
from elasticsearch.helpers import scan
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [55]:
# reading a dataframe

In [34]:
yyy2 = pd.read_csv('/Users/jahnavi/Documents/MyJob/Projects/Project_Recommender_System/data_for_similar_authors.csv')

In [36]:
yyy2['challenges_cat'] = yyy2.challenges_cat.apply(literal_eval)

In [37]:
yyy2['count_of'] = yyy2.count_of.apply(literal_eval)

----------------------------------*****************--------------------------
### Begin

In [56]:
def get_top5_users(user_id):    


    tags_to_be_found = yyy2[yyy2['userId']== user_id]['challenges_cat'].values[0] # tags for which the similar tags to be found

    list_to_be_found = yyy2[yyy2['userId']== user_id]['count_of'].values[0] # weights of the tags
    yyy3 = yyy2


    def get_score(tags_to_be_found,list_to_be_found,to_tested,list_tested):

        from sklearn.feature_extraction.text import CountVectorizer


        string1 = ' '.join(tags_to_be_found)
        string2 = ' '.join(to_tested)

        vectorizer = CountVectorizer()
        
# step to get all common words from both the strings and converting them to vectors

        matrix = vectorizer.fit_transform([string1, string2]) 

        x = vectorizer.transform([string1]) # transforming string1 to vectors
        y = vectorizer.transform([ string2]) # transforming string2 to vectors
        vectors = vectorizer.transform([string1, string2])

# we have got matrix for each vectors, now we have to add weights to that matrix
        total_weights = []
        lists = []
        lists.append(list_to_be_found)
        lists.append(list_tested)
        if len(lists[1]) == len(np.nonzero(matrix.toarray()[1])[0]):
            for k in range(matrix.toarray().shape[0]):
                l = lists[k]
                weights = np.zeros(matrix.toarray().shape[1])

                j=0


                for i in list(np.nonzero(matrix.toarray()[k])[0]):

                    weights[i] = l[j]

                    j = j+1
                total_weights.append(weights)

            a = x.toarray()*total_weights[0]
            b = y.toarray()*total_weights[1]

            eucledian_dist = np.sqrt(np.sum((a - b)**2))
        else:
            eucledian_disteucledian_dist = 0
        return eucledian_dist

    score_total = []
    for i in range(len(yyy3)):
        to_tested = yyy3['challenges_cat'][i]
        list_tested = yyy3['count_of'][i]
        score = get_score(tags_to_be_found,list_to_be_found,to_tested,list_tested)
        score_total.append(score)

    yyy3['score'] = pd.DataFrame(score_total)

    yyy3.rename(columns = {'tagged':'similarity_vector'}, inplace = True)
#     yyy3.rename(columns = {1:'rank'}, inplace = True)
#     print(yyy3.head())

    return yyy3.sort_values(by = 'score').iloc[1:][['userId','similarity_vector']].head()\
            .to_json(orient='records')

In [57]:
get_top5_users('00bd173704c04d4a8e2c05e43856e1b8')

'[{"userId":"8bc0343040004355b6a591c2b163f96f","similarity_vector":"[{\'Love\': 163.0}, {\'Marriage\': 163.0}, {\'Relationship\': 163.0}, {\'Strength\': 163.0}, {\'MentalHealth\': 95.0}]"},{"userId":"28587927aabf471db5314deb72eee554","similarity_vector":"[{\'HealthWellness\': 123.0}, {\'Love\': 123.0}, {\'MentalHealth\': 123.0}, {\'Pregnancy\': 123.0}, {\'Strength\': 123.0}]"},{"userId":"1c8455c67341490f90205180ad984c95","similarity_vector":"[{\'Love\': 220.0}, {\'MentalHealth\': 165.0}, {\'Relationship\': 112.0}, {\'Strength\': 112.0}, {\'Emotions\': 108.0}]"},{"userId":"797973e71828429bbf7e5310639d0386","similarity_vector":"[{\'Education\': 92.0}, {\'MentalHealth\': 92.0}, {\'Parenting\': 92.0}, {\'Motherhood\': 45.0}, {\'Love\': 20.0}]"},{"userId":"1635a89158434b2aaa36279b434abcf7","similarity_vector":"[{\'MentalHealth\': 178.0}, {\'Love\': 139.0}, {\'Education\': 109.0}, {\'SelfCare\': 91.0}, {\'Parenting\': 69.0}, {\'Marriage\': 56.0}, {\'Motherhood\': 56.0}]"}]'